# Training neural network model for the saucer dataset

### Import important libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Conv2D,MaxPool2D
import os

### Load an image and get its size

In [ ]:
img = image.load_img('saucer_data/train/normal/normal_3.jpg')
plt.imshow(img)
print(img.size)

### Dataset generation

In [ ]:
train = ImageDataGenerator(rescale= 1/255)
validation = ImageDataGenerator(rescale= 1/255)
train_dataset = train.flow_from_directory('saucer_data/train',target_size=(168,168),batch_size=24)
validation_dataset = train.flow_from_directory('saucer_data/validation',target_size=(168,168),batch_size=24)
print(train_dataset.class_indices)
print(validation_dataset.class_indices)
pred_dict = train_dataset.class_indices

### Train neural network model


In [ ]:
model = Sequential()
model.add(Conv2D(16,(3,3),activation='relu',input_shape=(168,168,3)))
print(model.output_shape)
model.add(MaxPool2D(2,2))
print(model.output_shape)
model.add(Conv2D(32,(3,3),activation='relu'))
print(model.output_shape)
model.add(MaxPool2D(2,2))
print(model.output_shape)
model.add(Conv2D(64,(3,3),activation='relu'))
print(model.output_shape)
model.add(MaxPool2D(2,2))
print(model.output_shape)
model.add(Conv2D(128,(3,3),activation='relu'))
print(model.output_shape)
model.add(MaxPool2D(2,2))
print(model.output_shape)
model.add(Flatten())
print(model.output_shape)
model.add(Dense(512,activation='relu'))
print(model.output_shape)
model.add(Dense(3,activation='softmax'))

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model_fit =model.fit(train_dataset,epochs=10,validation_data=validation_dataset)

### Check the models metrics

In [ ]:
f,ax=plt.subplots(2,1) #Creates 2 subplots under 1 column

#Assigning the first subplot to graph training loss and validation loss
ax[0].plot(model.history.history['loss'],color='g',label='Training Loss')
ax[0].plot(model.history.history['val_loss'],color='m',label='Validation Loss')
plt.legend()

#Plotting the training accuracy and validation accuracy
ax[1].plot(model.history.history['accuracy'],color='b',label='Training  Accuracy')
ax[1].plot(model.history.history['val_accuracy'],color='r',label='Validation Accuracy')

plt.legend()

### Test the model using testing dataset

In [ ]:
def get_key(val):
    for key, value in pred_dict.items():
         if val == value:
             return key
 
    return "key doesn't exist"

In [ ]:
dir_path = 'saucer_data/test'
for i in os.listdir(dir_path):
    print(i)
    img = image.load_img(dir_path+'//'+i)
    img = img.resize(((168,168)))
    plt.imshow(img)
    plt.show()

    X =  image.img_to_array(img)
    X = np.expand_dims(X,axis=0)
    images = np.vstack([X])
    val = model.predict_classes(images)
    print(get_key(val[0]))

### Save the model to json file and save its weights

In [ ]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")